In [ ]:
%pip install pygame

In [ ]:
from dataclasses import dataclass

import pygame
import random


SCREEN_HEIGHT, SCREEN_WIDTH = (160, 192)
SCREEN_CENTER_X, SCREEN_CENTER_Y = SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2
SCREEN_FACTOR = 4
WINDOW_TITLE = 'DeepPong'
WINDOW_ICON_FILEPATH = 'assets/favicon.ico'

SCORE_TO_WIN = 9
SCORE_PADDING = 10
SCORE_SIZE = 24
SCORE_FONT_FILEPATH = 'assets/font.ttf'
SCORE_SOUND_FILEPATH = 'assets/score.wav'
BOUNCE_SOUND_FILEPATH = 'assets/bounce.wav'

DASH_LENGTH = 4
DASH_WIDTH = 2
GAP_LENGTH = 8
GOAL_PADDING = 8


PADDLE_HEIGHT, PADDLE_WIDTH = (16, 2)
PADDLE_SPEED = 60

BALL_SIZE = 2
BALL_SPEED = 60

BLACK = (0, 0, 0)
WHITE = (255, 255, 255)

SOUND_VOLUME = 0.25
MILLISECONDS_PER_SECOND = 1000
MAX_DT = 1 / 30


def clamp(x: float, low: float, high: float):
    '''Clamps x to the range [low, high].'''
    return max(low, min(x, high))


def colliding(x1: float, y1: float, w1: float, h1: float, x2: float, y2: float, w2: float, h2: float) -> bool:
    '''Determines if two rectangles are overlapping based on the top left coordinate and their dimensions.'''
    return not (x1 + w1 <= x2 or x1 >= x2 + w2 or y1 - h1 >= y2 or y1 <= y2 - h2)


@dataclass(eq=False)
class Paddle:
    x: float
    y: float
    up: int
    down: int
    score: int = 0


class Ball:
    def __init__(self):
        self.x = SCREEN_CENTER_X - BALL_SIZE / 2
        self.y = SCREEN_CENTER_Y + BALL_SIZE / 2
        self.vx = BALL_SPEED * (-1) ** random.randint(0, 1)
        self.vy = BALL_SPEED * random.uniform(-1, 1)


class Pong:
    def __init__(self):
        if not pygame.get_init():
            pygame.init()

        self.clock = pygame.time.Clock()
        self.bounce = pygame.mixer.Sound(BOUNCE_SOUND_FILEPATH)
        self.score = pygame.mixer.Sound(SCORE_SOUND_FILEPATH)
        self.sounds = [self.bounce, self.score]
        self.mute()

        self.canvas = pygame.Surface((SCREEN_WIDTH, SCREEN_HEIGHT))
        self.font = pygame.font.Font(SCORE_FONT_FILEPATH, SCORE_SIZE)
        self.restart()

    def mute(self):
        '''Toggles sound for the game.'''
        for sound in self.sounds:
            volume = 0 if sound.get_volume() > 0 else SOUND_VOLUME
            sound.set_volume(volume)

    def update(self, dt):
        '''Updates the positions of all the game objects based on the amount of time that has passed since last frame (dt).'''
        if self.paused:
            return

        keys = pygame.key.get_pressed()

        # Move the paddles.
        for paddle in self.paddles:
            paddle.y += (keys[paddle.up] - keys[paddle.down]) * PADDLE_SPEED * dt
            paddle.y = clamp(paddle.y, PADDLE_HEIGHT, SCREEN_HEIGHT)

        # Update the ball's position.
        self.ball.y += self.ball.vy * dt
        self.ball.x += self.ball.vx * dt

        # Vertical bounce.
        if self.ball.y >= SCREEN_HEIGHT or self.ball.y <= BALL_SIZE:
            self.bounce.play()
            self.ball.vy *= -1

            if self.ball.y >= SCREEN_HEIGHT:
                self.ball.y = SCREEN_HEIGHT - (self.ball.y - SCREEN_HEIGHT)
            else:
                self.ball.y = abs(self.ball.y - BALL_SIZE)


        # Check for goals if the game is in progress.
        if self.playing:
            for paddle in self.paddles:
                if colliding(paddle.x, paddle.y, PADDLE_WIDTH, PADDLE_HEIGHT, self.ball.x, self.ball.y, BALL_SIZE, BALL_SIZE):
                    self.bounce.play()
                    self.ball.vx *= -1

                    if paddle == self.left:
                        self.ball.x += paddle.x + PADDLE_WIDTH - self.ball.x
                    else:
                        self.ball.x -= self.ball.x + BALL_SIZE - paddle.x

                    offset = (self.ball.y - BALL_SIZE / 2) - (paddle.y - PADDLE_HEIGHT / 2)
                    standardized = clamp(offset / (PADDLE_HEIGHT / 2), -1, 1)
                    self.ball.vy = BALL_SPEED * standardized

            # Check to see if someone scored.
            if self.ball.x <= 0:
                self.score.play()
                self.right.score += 1
                self.reset()
                self.ball.vx = abs(self.ball.vx)
            elif self.ball.x + BALL_SIZE >= SCREEN_WIDTH:
                self.score.play()
                self.left.score += 1
                self.reset()
                self.ball.vx = -abs(self.ball.vx)

        # Otherwise the game is over: let the ball bounce freely against the goals without the paddles.
        elif self.ball.x <= 0:
            self.bounce.play()
            self.ball.vx *= -1
            self.ball.x = abs(self.ball.x)
        elif self.ball.x + BALL_SIZE >= SCREEN_WIDTH:
            self.bounce.play()
            self.ball.vx *= -1
            self.ball.x = SCREEN_WIDTH - (self.ball.x + BALL_SIZE - SCREEN_WIDTH)

    def reset(self):
        '''Resets the paddle and ball positions after a goal.'''
        self.ball = Ball()

        if max(self.left.score, self.right.score) == SCORE_TO_WIN:
            self.playing = False

    def restart(self):
        '''Restarts the game entirely.'''
        self.ball = Ball()
        self.left = Paddle(GOAL_PADDING, (SCREEN_HEIGHT + PADDLE_HEIGHT) / 2, up=pygame.K_w, down=pygame.K_s)
        self.right = Paddle(SCREEN_WIDTH - (GOAL_PADDING + PADDLE_WIDTH), (SCREEN_HEIGHT + PADDLE_HEIGHT) / 2, up=pygame.K_UP, down=pygame.K_DOWN)
        self.paddles = [self.left, self.right]
        self.playing = True
        self.paused = True

    def render(self):
        '''Draws the frame to an internal surface.'''
        self.canvas.fill(BLACK)

        # Draw the paddles.
        if self.playing:
            for paddle in self.paddles:
                rectangle = pygame.Rect(paddle.x, SCREEN_HEIGHT - paddle.y, PADDLE_WIDTH, PADDLE_HEIGHT)
                pygame.draw.rect(self.canvas, WHITE, rectangle)

        # Draw the ball.
        rectangle = pygame.Rect(self.ball.x, SCREEN_HEIGHT - self.ball.y, BALL_SIZE, BALL_SIZE)
        pygame.draw.rect(self.canvas, WHITE, rectangle)

        # Draw the center divider.
        for y in range(0, SCREEN_HEIGHT, DASH_LENGTH + GAP_LENGTH):
            dash = pygame.Rect(SCREEN_CENTER_X - DASH_WIDTH / 2, y, DASH_WIDTH, DASH_LENGTH)
            pygame.draw.rect(self.canvas, WHITE, dash)

        # Draw the left paddle's score.
        score = self.font.render(str(self.left.score), False, WHITE)
        self.canvas.blit(score, ((SCREEN_WIDTH // 4) - (score.get_width() // 2), SCORE_PADDING))

        # Draw the right paddle's score.
        score = self.font.render(str(self.right.score), False, WHITE)
        self.canvas.blit(score, ((3 * SCREEN_WIDTH // 4) - (score.get_width() // 2), SCORE_PADDING))

    def show(self):
        '''Reveals the game in a desktop window.'''
        pygame.display.set_caption(WINDOW_TITLE)
        icon = pygame.image.load(WINDOW_ICON_FILEPATH)
        pygame.display.set_icon(icon)
        self.screen = pygame.display.set_mode((SCREEN_WIDTH * SCREEN_FACTOR, SCREEN_HEIGHT * SCREEN_FACTOR))

    def refresh(self):
        '''Refreshes the display.'''
        canvas = pygame.transform.scale(self.canvas, self.screen.get_size())
        self.screen.blit(canvas, (0, 0))
        pygame.display.flip()

    def run(self):
        '''Runs the game.'''
        running = True
        dt = 0.0

        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        running = False
                    elif event.key == pygame.K_SPACE:
                        self.paused = not self.paused
                    elif event.key == pygame.K_r:
                        self.restart()
                    elif event.key == pygame.K_m:
                        self.mute()

            self.update(dt)
            self.render()
            self.refresh()

            milliseconds = self.clock.tick(60)
            dt = milliseconds / MILLISECONDS_PER_SECOND
            dt = clamp(dt, 0, MAX_DT)

        pygame.quit()

In [ ]:
pong = Pong()
pong.show()
pong.run()